In [23]:
!pip install selenium
!pip install webdriver_manager


  Obtaining dependency information for webdriver_manager from https://files.pythonhosted.org/packages/b1/51/b5c11cf739ac4eecde611794a0ec9df420d0239d51e73bc19eb44f02b48b/webdriver_manager-4.0.1-py2.py3-none-any.whl.metadata


In [12]:
import requests
from bs4 import BeautifulSoup
from googlesearch import search
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import time
import pandas as pd

def google_search(query, num_results=10):
    """Perform a Google search and return a list of URLs."""
    return list(search(query, num=num_results))

def scrape_article(url, retries=3, timeout=5):
    for i in range(retries):
        try:
            response = requests.get(url, timeout=timeout)
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')
                title = soup.title.string if soup.title else 'No title'
                paragraphs = soup.find_all('p')
                content = ' '.join([para.get_text() for para in paragraphs])
                return {'title': title, 'content': content}
            else:
                print(f"Error: Received status code {response.status_code} for URL: {url}")
        except requests.exceptions.Timeout:
            print(f"Timeout occurred for URL: {url}. Retrying ({i+1}/{retries})...")
            time.sleep(2)  # Wait for 2 seconds before retrying
        except Exception as e:
            print(f"Error scraping {url}: {e}")
            break
    return None

def scrape_articles_for_ticker(ticker, num_results=10):
    """Search Google for articles related to a stock ticker and scrape the content."""
    query = f"{ticker} stock news"
    urls = google_search(query, num_results)
    articles = []
    for url in urls:
        article = scrape_article(url)
        if article and article['content']:
            articles.append(article)

    # save the articles to a CSV file
    df = pd.DataFrame(articles)
    df.to_csv(f'{ticker}_news.csv', index=False)
    
    return articles

articles = scrape_articles_for_ticker('MSFT', num_results=5)

Error: Received status code 503 for URL: https://www.tipranks.com/news/msft-googl-2-stocks-leading-the-ai-charge
Error: Received status code 503 for URL: https://www.tipranks.com/news/msft-googl-2-stocks-leading-the-ai-charge
Error: Received status code 503 for URL: https://www.tipranks.com/news/msft-googl-2-stocks-leading-the-ai-charge
Error: Received status code 401 for URL: https://www.marketwatch.com/investing/stock/msft
Error: Received status code 401 for URL: https://www.marketwatch.com/investing/stock/msft
Error: Received status code 401 for URL: https://www.marketwatch.com/investing/stock/msft
Error: Received status code 401 for URL: https://www.marketwatch.com/investing/stock/msft/analystestimates
Error: Received status code 401 for URL: https://www.marketwatch.com/investing/stock/msft/analystestimates
Error: Received status code 401 for URL: https://www.marketwatch.com/investing/stock/msft/analystestimates
Error: Received status code 401 for URL: https://www.marketwatch.com/i

[{'title': 'MSFT: Microsoft Corp - Stock Price, Quote and News - CNBC',
  'content': "Credit Cards Loans Banking Mortgages Insurance Credit Monitoring Personal Finance Small Business Taxes Help for Low Credit Scores Investing SELECT All Credit Cards Find the Credit Card for You Best Credit Cards Best Rewards Credit Cards Best Travel Credit Cards Best 0% APR Credit Cards Best Balance Transfer Credit Cards Best Cash Back Credit Cards Best Credit Card Welcome Bonuses Best Credit Cards to Build Credit SELECT All Loans Find the Best Personal Loan for You Best Personal Loans Best Debt Consolidation Loans Best Loans to Refinance Credit Card Debt Best Loans with Fast Funding Best Small Personal Loans Best Large Personal Loans Best Personal Loans to Apply Online Best Student Loan Refinance SELECT All Banking Find the Savings Account for You Best High Yield Savings Accounts Best Big Bank Savings Accounts Best Big Bank Checking Accounts Best No Fee Checking Accounts No Overdraft Fee Checking Acco

In [52]:
# save articles to a CSV file
df = pd.DataFrame(articles)
df.to_csv('ScrapedWebData/msft_news.csv', index=False)
df.head()


,0
0,


In [53]:
# Load the articles from the file
df = pd.read_csv('ScrapedWebData/msft_news.csv')

# Load the articles into a DataFrame
test_df = pd.DataFrame(df, columns=['title', 'content'])

## BERT Model Predictions:

### Pre-Trained FinBERT Model:

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

def preprocess_and_tokenize(articles):
    tokenized_articles = []
    for article in articles:
        content = article['content']
        tokens = tokenizer(content, padding='max_length', truncation=True, max_length=512, return_tensors='pt')
        tokenized_articles.append(tokens)
    
    return tokenized_articles

tokenized_articles = preprocess_and_tokenize(articles)

ConnectionError: HTTPSConnectionPool(host='cdn-lfs.huggingface.co', port=443): Read timed out.

In [1]:
import torch
from transformers import BertForSequenceClassification
import torch.nn.functional as F

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
    
def sentiment_analysis(tokenized_articles):
    sentiments = []
    with torch.no_grad():
        for tokens in tokenized_articles:
            outputs = model(**tokens)
            logits = outputs.logits
            probabilities = torch.nn.functional.softmax(logits, dim=-1)
            sentiment_score = (probabilities[0, 2], probabilities[0, 0])  # Positive probability - Negative probability
            sentiments.append(sentiment_score)
    
    return sentiments

sentiments = sentiment_analysis(tokenized_articles)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

NameError: name 'tokenized_articles' is not defined

In [52]:
def interpret_sentiments(articles, sentiments):
    for i, article in enumerate(articles):
        print(f"Title: {article['title']}")
        print(f"Sentiment Scores: Positive: {sentiments[i][0]}, Negative: {sentiments[i][1]}")
        print("="*50)

interpret_sentiments(articles, sentiments)

Title: Warren Buffett Is Dumping Apple Stock
Sentiment Scores: Positive: 0.23685304820537567, Negative: 0.2907799184322357
Title: AAPL: Apple Inc - Stock Price, Quote and News - CNBC
Sentiment Scores: Positive: 0.23258927464485168, Negative: 0.3506969213485718
Title: AAPL Stock Quote Price and Forecast | CNN
Sentiment Scores: Positive: 0.2847846448421478, Negative: 0.2933061122894287
Title: Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance
Sentiment Scores: Positive: 0.3126884698867798, Negative: 0.2793036699295044
Title: AAPL Interactive Stock Chart | Apple Inc. Stock - Yahoo Finance
Sentiment Scores: Positive: 0.26379743218421936, Negative: 0.38143816590309143
Title: Apple Stock Price | AAPL Stock Quote, News, and History | Markets Insider
Sentiment Scores: Positive: 0.24807433784008026, Negative: 0.2878495752811432
Title: Apple (AAPL) Stock Price, News & Info | The Motley Fool
Sentiment Scores: Positive: 0.2289879471063614, Negative: 0.31126168370246887
Title: Buy

### Fine Tuning of Standard BERT Model Using Labeled Financial Article Dataset:

In [16]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import DistilBertModel, DistilBertTokenizer
from transformers import BertModel, BertTokenizer
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import numpy as np
from collections import Counter
import random
from tqdm import tqdm

In [36]:
train_df = pd.read_csv('FinData/all-data.csv', encoding='latin-1', header=None, names=['sentiment', 'text'])
train_df['sentiment'] = train_df['sentiment'].map({'positive': 2, 'neutral': 1, 'negative': 0})

train_df.to_csv('ScrapedWebData/a-data.csv', index=False)

train_df.head()


,sentiment,text
0,1,"According to Gran , the company has no plans t..."
1,1,Technopolis plans to develop in stages an area...
2,0,The international electronic industry company ...
3,2,With the new production plant the company woul...
4,2,According to the company 's updated strategy f...


In [37]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 512
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 8
EPOCHS = 10
LEARNING_RATE = 1e-05

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [19]:
class TextData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        title = str(self.data.text[index])

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.sentiment[index], dtype=torch.long)
        }

    def __len__(self):
        return self.len

In [38]:
# Creating the dataset and dataloader for the neural network

train_size = 0.9
train_dataset=train_df.sample(frac=train_size,random_state=200)
valid_dataset=train_df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(train_df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("VALIDATION Dataset: {}".format(valid_dataset.shape))

training_set = TextData(train_dataset, tokenizer, MAX_LEN)
validating_set = TextData(valid_dataset, tokenizer, MAX_LEN)

FULL Dataset: (4846, 2)
TRAIN Dataset: (4361, 2)
VALIDATION Dataset: (485, 2)


In [39]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

valid_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
validating_loader = DataLoader(validating_set, **valid_params)

In [40]:
# Creating the customized model, by adding a drop out and a dense layer on top of BERT to get the final output for the model.
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = BertModel.from_pretrained("bert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, 3)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1.pooler_output
        hidden_state = self.pre_classifier(hidden_state)
        hidden_state = F.relu(hidden_state)
        hidden_state = self.dropout(hidden_state)
        output = self.classifier(hidden_state)
        return output

In [41]:
model = BERTClass()
device = torch.device('cuda' if torch.cuda.is_available() else 'mps')
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [28]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)

In [29]:
def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [32]:
data_path = 'Weights/'

# Defining the training function on the 90% of the dataset for tuning the distilbert model
def train(epochs, model, optimizer, loss_function, training_loader, validation_loader):
    losses = []
    accuracies = []
    for epoch in range(epochs):
      tr_loss = 0
      n_correct = 0
      nb_tr_steps = 0
      nb_tr_examples = 0
      model.train()

      for data in tqdm(training_loader, desc=f"Training epoch {epoch+1}"):
          ids = data['ids'].to(device, dtype = torch.long)
          mask = data['mask'].to(device, dtype = torch.long)
          targets = data['targets'].to(device, dtype = torch.long)

          outputs = model(ids, mask)
          loss = loss_function(outputs, targets)
          tr_loss += loss.item()
          big_val = torch.argmax(outputs.data, dim=1)
          n_correct += calcuate_accu(big_val, targets)

          nb_tr_steps += 1
          nb_tr_examples+=targets.size(0)

          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

      epoch_loss = tr_loss/nb_tr_steps
      epoch_accu = (n_correct)/nb_tr_examples

      model.eval()
      val_loss, val_correct, val_steps, val_examples = 0, 0, 0, 0
      with torch.no_grad():
          for data in tqdm(validation_loader, desc=f"Validating epoch {epoch+1}"):
              ids = data['ids'].to(device, dtype=torch.long)
              mask = data['mask'].to(device, dtype=torch.long)
              targets = data['targets'].to(device, dtype=torch.long)

              outputs = model(ids, mask)
              loss = loss_function(outputs, targets)
              val_loss += loss.item()
              big_val = torch.argmax(outputs, dim=1)
              val_correct += (big_val == targets).sum().item()

              val_steps += 1
              val_examples += targets.size(0)

      val_epoch_loss = val_loss / val_steps
      val_epoch_accu = val_correct / val_examples
      print(f'Epoch {epoch+1}/{epochs}: training loss = {epoch_loss:.3f}, validation loss = {val_epoch_loss:.3f}, Test Accuracy = {(val_epoch_accu)*100:.2f}%\n')
      torch.save(model.state_dict(), data_path + f'model_weights_{epoch}.pth')

In [34]:
train(2, model, optimizer, loss_function, training_loader, validating_loader)

Training epoch 1:   0%|          | 0/273 [00:00<?, ?it/s]/Applications/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Validating epoch 1: 100%|██████████| 61/61 [00:12<00:00,  5.05it/s]


Epoch 1/2: training loss = 1.133, validation loss = 1.121, Test Accuracy = 29.28%



Validating epoch 2: 100%|██████████| 61/61 [00:10<00:00,  6.00it/s]


Epoch 2/2: training loss = 1.131, validation loss = 1.122, Test Accuracy = 29.28%

